In [35]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.layers as layers
from sklearn.model_selection import KFold
import scipy as sc


import training_utils

In [36]:
img_size=(240, 320)
img_channels = 3
batch_size=32
epochs = 50
display_epochs = (0, 100)


In [37]:
model_name = 'Xception_like_global(32-64-126)2_(NaN).tf'

In [38]:
train_size = 1000
validation_size = 500
test_size = 500

# Load the DataSet

Dataset citation:
- From Semi-Supervised to Transfer Counting of Crowds
C. C. Loy, S. Gong, and T. Xiang
in Proceedings of IEEE International Conference on Computer Vision, pp. 2256-2263, 2013 (ICCV)
- Cumulative Attribute Space for Age and Crowd Density Estimation
K. Chen, S. Gong, T. Xiang, and C. C. Loy
in Proceedings of IEEE Conference on Computer Vision and Pattern Recognition, pp. 2467-2474, 2013 (CVPR, Oral)
- Crowd Counting and Profiling: Methodology and Evaluation
C. C. Loy, K. Chen, S. Gong, T. Xiang
in S. Ali, K. Nishino, D. Manocha, and M. Shah (Eds.), Modeling, Simulation and Visual Analysis of Crowds, Springer, vol. 11, pp. 347-382, 2013
- Feature Mining for Localised Crowd Counting
K. Chen, C. C. Loy, S. Gong, and T. Xiang
British Machine Vision Conference, 2012 (BMVC)

In [39]:
dataset_path = '/Users/olove/Library/CloudStorage/OneDrive-Personal/AI datasets/CrowdCounter'

In [40]:
labels_df = pd.read_csv(dataset_path + '/labels.csv')
labels_df['image_name'] = labels_df['id'].map('seq_{:06d}.jpg'.format)
labels_df.drop("id", axis=1,inplace=True)
display(labels_df)

,count,image_name
0,35,seq_000001.jpg
1,41,seq_000002.jpg
2,41,seq_000003.jpg
3,44,seq_000004.jpg
4,41,seq_000005.jpg
...,...,...
1995,27,seq_001996.jpg
1996,27,seq_001997.jpg
1997,25,seq_001998.jpg
1998,26,seq_001999.jpg


In [41]:
labels_df = labels_df.sample(frac=1).reset_index(drop=True)
display(labels_df)

,count,image_name
0,31,seq_001246.jpg
1,34,seq_001237.jpg
2,29,seq_001300.jpg
3,33,seq_000142.jpg
4,27,seq_001380.jpg
...,...,...
1995,32,seq_001560.jpg
1996,28,seq_000159.jpg
1997,30,seq_000815.jpg
1998,28,seq_000799.jpg


# Define the model

In [42]:
inputs = keras.Input(shape= img_size + (img_channels,))

## Custom Simple Covnet

Downsizing using strides instead of MaxPolling in order to conserve location data

In [43]:
def basic_covnet_block(input):
    x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu")(inputs)
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu")(x)
    x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu")(x)
    return x

## Custom Depthwise Seperable Convolution

In [44]:
def custom_depthwise_conv_block(input, sizes, repeat_per_size, stride = 1, pooling = False, dropout = False, residual = False):
    x = input

    previous_block_activation = x
    
    for size in sizes:
        for i in range(repeat_per_size - 1):
            x = layers.BatchNormalization()(x)
            x  = layers.Activation("relu")(x)
            x = layers.SeparableConv2D(size, 3, padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x  = layers.Activation("relu")(x)
        if dropout:
            x = layers.Dropout(0.5)(x)
        x = layers.SeparableConv2D(size, 3, strides=1 if pooling else stride, padding='same', use_bias=False)(x)
        if pooling:
            x = layers.MaxPooling2D(3, strides=stride, padding='same')(x)
        if residual:
            r = layers.SeparableConv2D(size, 1, strides=stride, padding="same")(
                previous_block_activation
            )
            x = layers.add([x, r])
            previous_block_activation = x



    return x
        

In [45]:
def vgg_16_model():
    covnet = keras.applications.vgg16.VGG16(
        include_top=False, 
        weights='imagenet', 
        input_shape=img_size + (img_channels,))(inputs)
    covnet.trainable = False
    outputs = basic_dense_block(covnet, [256])
    return keras.Model(inputs=inputs, outputs=outputs)

## Output Layer

In [46]:
def basic_dense_block(covnet_param, sizes, global_pooling = False):
    if global_pooling:
        x = layers.GlobalAveragePooling2D()(covnet_param)
    else:
        x = layers.Flatten()(covnet_param)
    x = layers.Dropout(0.5)(x)
    for size in sizes:
        x = layers.Dense(size, activation="relu")(x)
        x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1)(x)
    return outputs

In [47]:
def Xception_model():
    covnet = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(inputs)
    covnet = custom_depthwise_conv_block(covnet, [32, 64], 2, stride=2, pooling=True, residual=True)
    covnet = custom_depthwise_conv_block(covnet, [128], 2, residual=True)
    outputs = basic_dense_block(covnet, [], global_pooling=True)
    return keras.Model(inputs=inputs, outputs=outputs)

In [48]:
model = vgg_16_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 240, 320, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 10, 512)        14714688  
                                                                 
 flatten_2 (Flatten)         (None, 35840)             0         
                                                                 
 dropout_2 (Dropout)         (None, 35840)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               9175296   
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257 

# Train and compare different models

In [49]:
model.compile(loss="mse", optimizer="adam", metrics=["mae"])
# TODO: Try mae vs accuracy. mae should be better since we are adjusting it to get closer to the actual value

In [50]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=4
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name,
        monitor="val_loss",
        save_best_only=True
    ),
#    keras.callbacks.TensorBoard()
]

In [51]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 2)

result= next(kf.split(labels_df), None)
display(labels_df.iloc[result[0]])

test =  labels_df.iloc[result[1]]
display(test)
print(result[0])


,count,image_name
0,31,seq_001246.jpg
1,34,seq_001237.jpg
5,29,seq_001923.jpg
6,28,seq_001226.jpg
8,42,seq_000414.jpg
...,...,...
1992,31,seq_001443.jpg
1993,26,seq_000837.jpg
1994,28,seq_001378.jpg
1997,30,seq_000815.jpg


,count,image_name
2,29,seq_001300.jpg
3,33,seq_000142.jpg
4,27,seq_001380.jpg
7,37,seq_001623.jpg
37,27,seq_001355.jpg
...,...,...
1982,24,seq_001416.jpg
1983,28,seq_001699.jpg
1995,32,seq_001560.jpg
1996,28,seq_000159.jpg


[   0    1    5 ... 1994 1997 1999]


In [52]:
all(labels_df.iloc[result[0]]['image_name'].apply(lambda x: isinstance(x, str)))

True

In [53]:
def k_fold_validation(i_model):
    Wsave = i_model.get_weights()

    i_kf = KFold(n_splits = 5, shuffle = True, random_state = 2)
    
    history_store = []
    
    for i_result in i_kf.split(labels_df):
    
        train = labels_df.iloc[i_result[0]]
        test =  labels_df.iloc[i_result[1]]
        
        print(i_result[0])
        print(i_result[1])
        
        display(labels_df)
        display(train)
        print(f'Train size: {len(train)}')
        display(test)
        print(f'Test size: {len(test)}')

        (train_generator, validation_generator) = training_utils.load_generators(train, test, dataset_path, batch_size, img_size)

        i_model.set_weights(Wsave)
        i_history = i_model.fit(train_generator,
                                epochs=epochs,
                                callbacks = callbacks_list,
                                validation_data=validation_generator,
                                )
        
        history_store.append(i_history)
        
    return history_store
    

In [54]:
history = k_fold_validation(model)

[   0    1    5 ... 1994 1997 1999]
[   2    3    4    7   37   45   52   61   65   66   68   69   76   80
   82   83   84   85   93  101  112  114  117  119  123  137  142  143
  145  154  155  157  161  173  176  180  181  197  199  215  226  230
  231  232  236  239  240  241  246  248  263  268  270  271  275  278
  280  284  288  294  305  316  323  332  344  352  354  355  356  361
  365  368  370  372  376  385  391  401  411  418  426  445  448  452
  455  456  458  459  465  468  470  472  479  481  484  488  492  494
  501  502  503  505  515  524  526  529  538  540  547  556  557  558
  560  570  577  580  581  582  586  598  600  606  610  611  615  619
  625  627  633  642  643  652  662  675  677  681  694  695  711  728
  735  738  748  768  770  776  786  802  821  823  825  826  835  837
  840  844  849  851  852  871  874  880  882  888  891  897  905  909
  916  923  928  945  951  952  961  969  978  980  981  985  991  996
 1006 1014 1023 1025 1040 1042 1046 1051 

,count,image_name
0,31,seq_001246.jpg
1,34,seq_001237.jpg
2,29,seq_001300.jpg
3,33,seq_000142.jpg
4,27,seq_001380.jpg
...,...,...
1995,32,seq_001560.jpg
1996,28,seq_000159.jpg
1997,30,seq_000815.jpg
1998,28,seq_000799.jpg


,count,image_name
0,31,seq_001246.jpg
1,34,seq_001237.jpg
5,29,seq_001923.jpg
6,28,seq_001226.jpg
8,42,seq_000414.jpg
...,...,...
1992,31,seq_001443.jpg
1993,26,seq_000837.jpg
1994,28,seq_001378.jpg
1997,30,seq_000815.jpg


Train size: 1600


,count,image_name
2,29,seq_001300.jpg
3,33,seq_000142.jpg
4,27,seq_001380.jpg
7,37,seq_001623.jpg
37,27,seq_001355.jpg
...,...,...
1982,24,seq_001416.jpg
1983,28,seq_001699.jpg
1995,32,seq_001560.jpg
1996,28,seq_000159.jpg


...

Train size: 1600


,count,image_name
8,42,seq_000414.jpg
9,28,seq_001788.jpg
19,30,seq_000553.jpg
31,46,seq_001179.jpg
34,28,seq_000642.jpg
...,...,...
1981,25,seq_000522.jpg
1985,35,seq_000150.jpg
1987,27,seq_001996.jpg
1992,31,seq_001443.jpg


Test size: 400


,count,image_name
0,31,seq_001246.jpg
1,34,seq_001237.jpg
2,29,seq_001300.jpg
3,33,seq_000142.jpg
4,27,seq_001380.jpg
...,...,...
1995,32,seq_001560.jpg
1996,28,seq_000159.jpg
1997,30,seq_000815.jpg
1998,28,seq_000799.jpg


image_name
count
Found 1600 validated image filenames.
Found 400 validated image filenames.
Epoch 1/50
50/50 [==============================] - 70s 1s/step - loss: 74311974414199667294208.0000 - mae: 153821954048.0000 - val_loss: 58421243805404613312512.0000 - val_mae: 241470488576.0000
Epoch 2/50
50/50 [==============================] - 70s 1s/step - loss: 290267948738322232246272.0000 - mae: 434873335808.0000 - val_loss: 136410537919741484335104.0000 - val_mae: 369104551936.0000
Epoch 3/50
50/50 [==============================] - 70s 1s/step - loss: 393167093744408798953472.0000 - mae: 511796576256.0000 - val_loss: 144663294164905370845184.0000 - val_mae: 380086747136.0000
Epoch 4/50
50/50 [==============================] - 70s 1s/step - loss: 427613830257473040154624.0000 - mae: 532335722496.0000 - val_loss: 160102606465050943160320.0000 - val_mae: 399848964096.0000
Epoch 5/50
50/50 [==============================] - 70s 1s/step - loss: 469172831446065863458816.0000 - mae: 557781549

# Represent history

In [62]:
print (history)
mea_results = [np.min(x.history['mae']) for x in history]
val_mea_results = [np.min(x.history['val_mae']) for x in history]
loss_results = [np.min(x.history['loss']) for x in history]
val_loss_results = [np.min(x.history['val_loss']) for x in history]
print(mea_results)

[<keras.src.callbacks.History object at 0x2dd484a10>, <keras.src.callbacks.History object at 0x2c69031d0>, <keras.src.callbacks.History object at 0x29847a0d0>, <keras.src.callbacks.History object at 0x2984ebd90>, <keras.src.callbacks.History object at 0x2da0d1210>]
[66.37339782714844, 11.2545747756958, 444855025664.0, 2505.579345703125, 153821954048.0]


In [63]:
print(f'Mean mae: {np.mean(mea_results)}')
print(f'Mean val_mae: {np.mean(val_mea_results)}')
print(f'Mean loss: {np.mean(loss_results)}')
print(f'Mean val_loss: {np.mean(val_loss_results)}')

Mean mae: 119735396459.04146
Mean val_mae: 58102305424.88722
Mean loss: 1.1046698391200082e+23
Mean val_loss: 1.2172599964374565e+22


In [57]:
print(history[0].history.keys())

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


In [ ]:
plt.plot(history.history['mae'][display_epochs[0]:display_epochs[1]])
plt.plot(history.history['val_mae'][display_epochs[0]:display_epochs[1]])
plt.title('model mean squared')
plt.ylabel('mean squared')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'][display_epochs[0]:display_epochs[1]])
plt.plot(history.history['val_loss'][display_epochs[0]:display_epochs[1]])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Bibliografia

- Deep Learning with Python, Second Edition. François Chollet